### import

In [ ]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as mcolors
from matplotlib import rcParams
rcParams['text.usetex'] = True
rcParams['font.family'] = 'serif'
rcParams['font.serif'] = ['Times New Roman']

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import classification_report
from torchmetrics.functional import r2_score
from torchmetrics import AUROC
from torchmetrics.functional import accuracy
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score, roc_curve, auc, confusion_matrix, accuracy_score
from sklearn import metrics
from torchmetrics import F1Score
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
from torch.nn.utils import clip_grad_norm_
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import torch.nn.functional as F
import torch
import torch.nn as nn
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from transformers import get_linear_schedule_with_warmup
from transformers import AdamW
from pytorch_lightning import Trainer
from transformers import AutoModel, AutoTokenizer
from transformers import BertTokenizerFast as BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup
import pytorch_lightning as pl
from torchmetrics.classification import F1Score
from sklearn.metrics import precision_recall_fscore_support
import torch.nn.functional as F
from sklearn.metrics import classification_report
import torchmetrics
from torchmetrics import Metric
import torchmetrics.classification
import re
from pytorch_lightning import LightningModule
from pytorch_lightning.trainer import Trainer
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from pytorch_lightning.callbacks import LearningRateMonitor
from torchmetrics import Accuracy, MetricCollection, Precision, Recall, F1Score, Metric
from sklearn.metrics import confusion_matrix
from torchmetrics.classification import BinaryF1Score, BinaryConfusionMatrix
from torch import tensor
from typing import Optional
from pytorch_lightning.callbacks import Callback
import time
import numpy as np
from pytorch_lightning.callbacks import LearningRateMonitor
from pytorch_lightning import Trainer, seed_everything
from torchmetrics import Accuracy, MetricCollection, Precision, Recall, ROC
from torch import tensor
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score, roc_curve
from pytorch_lightning.callbacks import Callback

In [ ]:
train_df_color = '#E9BFC0'          #red
val_df_color = '#C3D5E7'            #blue
test_df_color = '#F4D98F'           #yellow
threshold_color = '#DC96AE'         #pink

real_line_color = '#93c47d'         #blue
fake_line_color = '#E9BFC0'         #red
maybe_line_color = '#F4D98F'        #yellow

real_dot_color = '#56943a'          #green hard
fake_dot_color = '#8a292c'          #red red
maybe_dot_color = '#d69d00'         #yellow hard

logreg_line_color = '#7db364'       #green
logreg_fill_color = '#97c283'
logreg_dot_color = '#48802e'

binary_line_color = '#aeb1cf'       #purple
binary_dot_color = '#8a8db6'
binary_fill_color = '#bec0d8'       #3 tint lighter

multiclass_line_color = '#a2c9ef'   #blue
multiclass_fill_color = '#7a8c9f'
multiclass_dot_color = '#1e456b' 

colors = ['#d1e5f0', '#94c4f3', '#63a7e0', '#358ece', '#0a6ab7', '#084f8a'] #blues for CM
cmap = sns.color_palette(colors, as_cmap=True)

In [ ]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r"@\w*", " ", text)                                           #tags
    text = re.sub(r"#|^\*|\*$|&quot;|&gt;|&lt;|&lt;3", " ", text)               #special characters
    text = text.replace("&amp;", " and ")                                       #&
    text = re.sub(r"ht+p+s?://\S*", " ", text)                                  #links
    text = re.sub(r"[^a-zA-Z0-9æøåÆØÅ\d\s:/£$%€,-]|(?<!\S)å(?!\S)", "", text)   #ascii
    text = re.sub(r",(?!\d)", "", text)                                         #comma
    text = re.sub(r"(\S)\.(\S)", r"\1. \2", text)                               #period
    text = re.sub(r"\(\)|\[\]|\{\}", " ", text)                                 #brackets
    text = re.sub(r"\s[b-gjz]\s", " ", text)                                    #single chars
    text = " ".join(text.split()).strip()                                       #spaces
    return text

In [ ]:
RANDOMSEED = 42
seed_everything(RANDOMSEED, workers=True) #for dataloaders
torch.set_float32_matmul_precision('medium')

In [ ]:
train_df = pd.read_csv('/home/thombras/files/NorLIAR_train_bin.csv')
val_df = pd.read_csv('/home/thombras/files/NorLIAR_val_bin.csv')
test_df = pd.read_csv('/home/thombras/files/NorLIAR_test_bin.csv')
PFO_test_df = pd.read_csv('/home/thombras/files/PFO.csv')
FND_test_df = pd.read_csv('/home/thombras/files/FND_gossip_df.csv')
LABEL_COLUMNS = train_df.columns.tolist()[6:]

In [ ]:
train_df['claim_no'] = train_df['claim_no'].apply(preprocess_text)
val_df['claim_no'] = val_df['claim_no'].apply(preprocess_text)
test_df['claim_no'] = test_df['claim_no'].apply(preprocess_text)
PFO_test_df['claim_no'] = PFO_test_df['claim_no'].apply(preprocess_text)
FND_test_df['claim_no'] = FND_test_df['claim_no'].apply(preprocess_text)
test_df.shape, PFO_test_df.shape, FND_test_df.shape, LABEL_COLUMNS

### norbert3

In [ ]:
BASE_MODEL = 'ltg/norbert3-base' #LTG = Language Technology Group (University of Oslo)
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
norbert3 = AutoModel.from_pretrained(BASE_MODEL, trust_remote_code=True, output_hidden_states=True)

### encoding

In [ ]:
BATCH_SIZE = 32
MAX_TOKEN_COUNT = 60
N_EPOCHS = 20
N_WORKERS = 2
INPUT_SIZE = norbert3.config.hidden_size 

class data_encode(Dataset):
    def __init__(self, data: pd.DataFrame, 
                tokenizer: AutoTokenizer, 
                MAX_TOKEN_COUNT: int = 128,
                labels=None): 
        self.tokenizer = tokenizer
        self.data = data
        self.max_token_len = MAX_TOKEN_COUNT
    
    def __len__(self):                  
        return len(self.data)           
    
    def __getitem__(self, index: int):  
        data_row = self.data.iloc[index]
        text = data_row.claim_no
        labels = data_row[LABEL_COLUMNS] 
        encoding = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_token_len,
            return_token_type_ids=False,
            padding="max_length",  
            truncation=True,
            return_attention_mask=True, 
            return_tensors='pt', 
        )
        return dict(
            text=text,
            input_ids=encoding["input_ids"].flatten(),       
            attention_mask=encoding["attention_mask"].flatten(),
            labels=torch.FloatTensor(labels))  

In [ ]:
train_dataset = data_encode(
    train_df,
    tokenizer,
    MAX_TOKEN_COUNT,
    LABEL_COLUMNS)

val_dataset = data_encode(
    val_df,
    tokenizer,
    MAX_TOKEN_COUNT,
    LABEL_COLUMNS)

test_dataset = data_encode(
    test_df,
    tokenizer,
    MAX_TOKEN_COUNT,
    LABEL_COLUMNS)

PFO_test_dataset = data_encode(
    PFO_test_df,
    tokenizer,
    MAX_TOKEN_COUNT,
    LABEL_COLUMNS)

FND_test_dataset = data_encode(
    FND_test_df,
    tokenizer,
    MAX_TOKEN_COUNT,
    LABEL_COLUMNS)

train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=N_WORKERS)
val_loader = DataLoader(dataset=val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=N_WORKERS)
test_loader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=N_WORKERS)
PFO_test_loader = DataLoader(dataset=PFO_test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=N_WORKERS)
FND_test_loader = DataLoader(dataset=FND_test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=N_WORKERS)

In [ ]:
steps_per_epoch = len(train_df) // BATCH_SIZE  #binary floor division
total_training_steps = steps_per_epoch * N_EPOCHS
WARMUP_STEPS = total_training_steps // 5
TRAINING_STEPS = steps_per_epoch * N_EPOCHS - WARMUP_STEPS

print('Epochs:{0:15}'.format(N_EPOCHS), '\nBatch size:{0:12}'.format(BATCH_SIZE))
print('-----------------------------')
print('Dataset len:{0:13}'.format(len(train_dataset)))
print('Steps per epoch:{0:8}'.format(steps_per_epoch))
print('-----------------------------')
print('Total training steps:{0:8}'.format(total_training_steps))
print('Warmup is 1/5 of the total training steps:')
print('- warmup steps:{0:8}'.format(WARMUP_STEPS), '\n- training steps:{0:8}'.format(TRAINING_STEPS))

### architecture

In [ ]:
from torch.utils.tensorboard import SummaryWriter

class NB3BinaryArch(pl.LightningModule):              #https://torchmetrics.readthedocs.io/_/downloads/en/v0.7.1/pdf/
   def __init__(self, base_model, 
      input_layer=768, fc1=512, fc2=256, fc3=64,
      drop_rate=0.2, n_training_steps=None, n_warmup_steps=None, 
      lr=1e-5, wd=1e-2, eps=1e-8):
      super().__init__()
      self.drop_rate = drop_rate
      self.N_WARMUP_STEPS = n_warmup_steps
      self.N_TRAINING_STEPS = n_training_steps
      self.LR = lr
      self.WD = wd
      self.EPS = eps
      self.save_hyperparameters(ignore=['base_model']) #save params
      self.bert = base_model
      self.classification_head = nn.Sequential(
         nn.Linear(input_layer, fc1), nn.ReLU(), nn.Dropout(self.drop_rate),
         nn.Linear(fc1, fc2), nn.ReLU(), nn.Dropout(self.drop_rate),
         nn.Linear(fc2, fc3), nn.ReLU(), nn.Dropout(self.drop_rate),
         nn.Linear(fc3, 1))
      self.loss_func = nn.BCELoss()
      self.acc = torchmetrics.classification.Accuracy(task='binary', average='weighted').cuda()
      self.prec = torchmetrics.classification.Precision(task='binary', average='weighted').cuda()
      self.rec = torchmetrics.classification.Recall(task='binary', average='weighted').cuda()
      self.f1 = torchmetrics.classification.F1Score(task='binary', average='weighted').cuda()
      self.auroc = torchmetrics.classification.AUROC(task='binary').cuda()
      self.confusion_matrix = torchmetrics.classification.ConfusionMatrix(task='binary').cuda()
      self.roc = torchmetrics.classification.ROC(task='binary').cuda()
      self.val_y_prob = [] 
      self.val_labels = []
      self.test_y_prob = []
      self.test_labels = []
      self.writer = SummaryWriter(log_dir="/home/thombras/light_logs/binary", filename_suffix='bin_histo')
    
   def forward(self, input_ids, attention_mask, labels=None):
      output = self.bert(input_ids, attention_mask=attention_mask)
      pooler = output.last_hidden_state[:, 0, :] #CLS
      pooler_numpy = pooler.detach().cpu().numpy()
      self.writer.add_histogram('Pooler Output', pooler_numpy, self.global_step)
      logits = self.classification_head(pooler)
      logits_numpy = logits.detach().cpu().numpy()
      self.writer.add_histogram('Logits', logits_numpy, self.global_step)
      y_prob = torch.sigmoid(logits)
      y_prob_numpy = y_prob.detach().cpu().numpy()
      self.writer.add_histogram('Probabilities', y_prob_numpy, self.global_step) 
      return logits, y_prob

   def _recurrent_step(self, batch, batch_idx):
      input_ids = batch["input_ids"]
      attention_mask = batch["attention_mask"]
      labels = batch["labels"]
      logits, y_prob = self.forward(input_ids, attention_mask, labels=labels)
      loss = self.loss_func(y_prob, labels)
      return loss, logits, labels, y_prob
   
   def _calculate_metrics(self, batch, batch_idx):
      loss, logits, labels, y_prob = self._recurrent_step(batch, batch_idx)
      acc = self.acc(y_prob, labels)
      prec = self.prec(y_prob, labels)
      rec = self.rec(y_prob, labels)
      f1 = self.f1(y_prob, labels)
      return acc, prec, rec, f1

   def training_step(self, batch, batch_idx):
      loss, _, labels, y_prob = self._recurrent_step(batch, batch_idx)
      acc, prec, rec, f1 = self._calculate_metrics(batch, batch_idx)
      self.log_dict({'train_accuracy': acc, 'train_precision': prec, 'train_recall': rec, 'train_f1': f1},
               prog_bar=False, logger=True, on_step=True, on_epoch=True)
      self.log('train_loss', loss,  prog_bar=True, logger=True, on_step=True, on_epoch=True)
      return {'loss': loss, 'prediction': y_prob, 'labels': labels}

   def on_training_epoch_end(self):
      pass

   def on_train_end(self):
      self.writer.close()

   def validation_step(self, batch, batch_idx):
      loss, _, labels, y_prob = self._recurrent_step(batch, batch_idx)
      acc, prec, rec, f1 = self._calculate_metrics(batch, batch_idx)
      self.val_y_prob.append(y_prob)
      self.val_labels.append(labels)
      self.log_dict({'val_accuracy': acc, 'val_f1': f1},
               prog_bar=False, logger=True, on_step=True, on_epoch=True)
      self.log('val_loss', loss,  prog_bar=True, logger=True, on_step=True, on_epoch=True)
      return loss

   def on_validation_epoch_end(self):
      pass

   def on_validation_end(self):
      pass

   def test_step(self, batch, batch_idx):
      loss, _, labels, y_prob  = self._recurrent_step(batch, batch_idx)
      acc, prec, rec, f1 = self._calculate_metrics(batch, batch_idx)
      self.test_y_prob.append(y_prob)
      self.test_labels.append(labels)
      self.log_dict({'test_accuracy': acc, 'test_precision': prec, 'test_recall': rec, 'test_f1': f1},
               prog_bar=True, logger=True, on_step=True, on_epoch=True)
      self.log('test_loss', loss,  prog_bar=True, logger=True, on_step=True, on_epoch=True)
      return loss

   def on_test_end(self):
      test_y_prob_total = torch.cat(self.test_y_prob, dim=0)
      test_labels_total = torch.cat(self.test_labels, dim=0).long()
      print('test_y_prob_total\n', test_y_prob_total)
      print('test_labels_total\n', test_labels_total)
      
      #ROC
      self.roc.reset()
      self.roc.update(test_y_prob_total, test_labels_total)
      fpr, tpr, thresholds = self.roc.compute()
      fpr_cpu = fpr.cpu().numpy()
      tpr_cpu = tpr.cpu().numpy()
      best_threshold_index = np.argmax(tpr_cpu - fpr_cpu)
      plt.figure(figsize=(8, 5))
      plt.plot(fpr_cpu, tpr_cpu, color=binary_line_color, lw=2, label='ROC curve')
      plt.plot([0, 1], [0, 1], color='gray', lw=1, linestyle='--', label='Reference Line')
      plt.plot(fpr_cpu[best_threshold_index], tpr_cpu[best_threshold_index], 'o', color=binary_dot_color, label=f'Best Threshold: {thresholds[best_threshold_index]:.2f}')
      roc_treshold = thresholds[best_threshold_index]
      plt.xlabel('False Positive Rate', fontsize=12)
      plt.ylabel('True Positive Rate', fontsize=12)
      plt.title('ROC for Fake-NorBERT', fontsize=15)
      plt.legend(loc="lower right")
      print("FPR, TPR", fpr_cpu[best_threshold_index], tpr_cpu[best_threshold_index])
      print("Best threshold index", thresholds[best_threshold_index])
      plt.show()

      #AUROC
      test_y_prob_total_np = test_y_prob_total.cpu().numpy() #GPU to CPU + Tensor to Array
      test_labels_total_np = test_labels_total.cpu().numpy()
      fpr, tpr, thresholds = roc_curve(test_labels_total_np, test_y_prob_total_np)
      best_threshold = thresholds[np.argmax(tpr - fpr)]
      auroc = roc_auc_score(test_labels_total_np, test_y_prob_total_np)
      plt.figure(figsize=(8, 5))
      plt.plot(fpr, tpr, color=binary_line_color, lw=2, label='ROC curve')
      plt.fill_between(fpr, tpr, color=binary_fill_color, alpha=0.3) #shade the area under the curve
      plt.text(0.5, 0.5, f'AUROC = {auroc:.2f}', fontsize=13, ha='right', va='top')

      plt.xlabel('False Positive Rate', fontsize=12)
      plt.ylabel('True Positive Rate', fontsize=12)
      plt.title('AUROC for Fake-NorBERT', fontsize=15)
      plt.legend(loc="upper left")
      plt.show()
      print("AUROC value:", auroc)

      #CM
      print('thresh', roc_treshold)
      preds = (test_y_prob_total > roc_treshold).int()
      self.confusion_matrix.reset()
      test_end_cm = self.confusion_matrix(preds, test_labels_total)
      test_end_cm_np = test_end_cm.cpu().numpy()
      print("CM:", test_end_cm)
      
      labels = np.array([
         [f'True Negative\n{test_end_cm_np[0][0]}', f'False Positive\n{test_end_cm_np[0][1]}'],
         [f'False Negative\n{test_end_cm_np[1][0]}', f'True Positive\n{test_end_cm_np[1][1]}']])

      labels = np.asarray(labels).reshape(2, 2)
      plt.figure(figsize=(8, 5))
      sns.heatmap(test_end_cm_np, annot=labels, cmap=cmap, fmt="", annot_kws={"size": 13})
      plt.ylabel("True Class", fontsize=12)
      plt.xlabel("Predicted Class", fontsize=12)
      plt.title("CM for Fake-NorBERT", fontsize=15)
      plt.show()

      test_y_prob_total_cpu = test_y_prob_total.cpu()
      test_labels_total_cpu = test_labels_total.cpu()
      print('best_threshold',best_threshold)
      predictions = (test_y_prob_total_cpu > best_threshold).int()
      target_names = ['Fake', 'Real']
      print(classification_report(test_labels_total_cpu, predictions, target_names=target_names))

      self.test_y_prob = []
      self.test_labels = []

   def configure_optimizers(self):
      optimizer = torch.optim.AdamW(self.classification_head.parameters(), 
                        lr=self.LR, weight_decay=self.WD, eps=self.EPS)
      scheduler = get_linear_schedule_with_warmup(
         optimizer, 
         num_warmup_steps=self.N_WARMUP_STEPS, 
         num_training_steps=self.N_TRAINING_STEPS)
      return dict(optimizer=optimizer, 
         lr_scheduler=dict(scheduler=scheduler, 
                           interval='step'))
   
class TrainingTime(Callback):
    def on_train_start(self, trainer, NB3RegressorArch):
        self.start = time.time()
    
    def on_train_end(self, trainer, NB3RegressorArch):
        self.end = time.time()
        total_minutes = (self.end-self.start)
        print(f'\nTraining finished: {total_minutes:.2f} seconds\n') 

### fine-tune

In [ ]:
MONITOR_METRIC = 'val_loss'
MON_MODE = 'min'

MAX_EPOCHS = 20
FC1=1024
FC2=768
FC3=512
TUNER_LR = 1.15e-5
LR = TUNER_LR    
WD = 1e-2
EPS = 1e-8
DR = 0.2
DEVICE = 'gpu'
PATIENCE = 1
TOP_K = 1
DEV_RUN = 0

TB_LOGGER = TensorBoardLogger('/home/light_logs',
            name='bin_nn')

checkpoint_callback = ModelCheckpoint(
  dirpath = '/home/checkpoints',
  filename = 'best_binary',
  save_top_k = TOP_K, 
  verbose = True,
  monitor = MONITOR_METRIC,   
  mode = MON_MODE)

LR_MONITOR = LearningRateMonitor(logging_interval='epoch')

early_stopping_callback = EarlyStopping(
    monitor=MONITOR_METRIC, 
    patience=PATIENCE) 

nb3 = norbert3
binary_model = NB3BinaryArch(
    nb3,
    input_layer=INPUT_SIZE,
    fc1=FC1,
    fc2=FC2,
    fc3=FC3,
    drop_rate=DR,
    n_warmup_steps=WARMUP_STEPS,
    n_training_steps=TRAINING_STEPS,
    lr=LR,
    wd=WD,
    eps=EPS)

trainer = pl.Trainer(
    fast_dev_run=DEV_RUN,
    logger=TB_LOGGER,
    min_epochs=1,
    max_epochs=N_EPOCHS,
    accelerator = DEVICE,
    callbacks=[checkpoint_callback, 
               TrainingTime(), LR_MONITOR] 
)

print("\nFast_dev_run =", trainer.fast_dev_run)
print("Max epoch:", N_EPOCHS)
print("Batch size:", BATCH_SIZE)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
trainer.fit(binary_model, train_dataloaders=train_loader, val_dataloaders=val_loader)

In [ ]:
trainer.validate(binary_model, val_loader)

In [ ]:
m_version = '/home/checkpoints/best_binary.ckpt'
trainer.test(ckpt_path=m_version, dataloaders=test_loader)
#test_loader
#PFO_test_loader
#FND_test_loader

### predictions

In [ ]:
m_version = '/home/checkpoints/best_binary.ckpt'
Fake_NorBERT = NB3BinaryArch.load_from_checkpoint(checkpoint_path=m_version, base_model=norbert3)
Fake_NorBERT.eval()
Fake_NorBERT.freeze()
THRESH = 0.45   #from roc
MAX_TOKEN = 60

class FakeNorBERT_predict:      
    def __init__(self, model, tokenizer, max_token_len, threshold):
        self.MODEL = model
        self.TOKENIZER = tokenizer
        self.DEVICE = torch.device('cuda')
        self.MAX_TOKEN_LEN = max_token_len
        self.THRESH = threshold

    def calculate(self, text):
        encode = self.TOKENIZER.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.MAX_TOKEN_LEN,
            return_token_type_ids=False,
            padding="max_length",
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt')
        input_ids = encode['input_ids'].to(self.DEVICE)
        attention_mask = encode['attention_mask'].to(self.DEVICE)
        with torch.no_grad():
            outputs = self.MODEL(input_ids=input_ids, attention_mask=attention_mask)
            unnormalized = outputs[0].item()
            normalized = outputs[1].item()
        return unnormalized, normalized

    def score(self, text):
        _, normalized = self.calculate(text)
        print(round(normalized, 3))

    def logit(self, text):
        unnormalized, _ = self.calculate(text)
        print(round(unnormalized, 3))

    def predict(self, text):
        _, normalized = self.calculate(text)
        if normalized < self.THRESH:
            print("Fake")
        else:
            print("Real")

FNB = FakeNorBERT_predict(model=Fake_NorBERT, tokenizer=tokenizer, max_token_len=MAX_TOKEN, threshold=THRESH)

In [ ]:
i = 0
test = test_dataset[i]['text']
FNB.score(test)
FNB.predict(test)

### shap

In [ ]:
import shap

tokenizer_pattern = r"\W+"
def shap_prediction(texts): #wrapper for shap
    normalized_preds = []
    for text in texts:
        _, normalized = FNB.calculate(text)
        normalized_preds.append(normalized)
    return normalized_preds

masker = shap.maskers.Text(tokenizer=tokenizer_pattern)
explainer = shap.Explainer(shap_prediction, masker)
shap_values = explainer([test])

shap.plots.text(shap_values) 

In [ ]:
#other visualisation plots
shap.initjs()
shap.plots.force(shap_values[0],plot_cmap=[real_dot_color,fake_dot_color])
shap.waterfall_plot(shap.Explanation(values=shap_values[0]))